# core

> Basic primitives for composing signals

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from mana_signals.reactive import *
from fastcore.all import *
import numpy as np
from sklearn import linear_model

In [ ]:
#| export
class RDelay(RValue):
    """Value shifted from the calculation by given delay"""
    def __init__(self, source:RValue,  # source value
                 delay_n:int=1 # amount of shift
                ): 
        super().__init__()
        store_attr();
        self.history = np.full(delay_n+1, np.nan)
    def calc(self): 
        self.history = np.roll(self.history, shift=1)
        self.history[0] = self.source.value
        return self.history[-1]
        
        

Example of using shifted reactive value

In [ ]:
a = RInput(0)
v = RDelay(a, delay_n=1)
print(v.value)
a.set_value(8);print(v.value)
a.set_value(1);print(v.value)

nan
0.0
8.0


In [ ]:
a=np.array([1,2,3])
np.roll(a, shift=1)

array([3, 1, 2])

In [ ]:
#| export
class RLastn(RValue): 
    """Last n values collected from the given source"""
    def __init__(self, source: RValue, # source value
                 n:int = 1, # number of values to collect
                 missing_val = np.nan # filler value for missing data
                ): 
        super().__init__()
        self.source,self.n = source,n
        self.lastn = np.full(n, missing_val)
    def calc(self): 
        self.lastn = np.roll(self.lastn, shift=1)
        self.lastn[0] = self.source.value
        return self.lastn

In [ ]:
a = RInput(0)
v = RLastn(a, 3)
for i in range(5): 
    print(v.value)
    a.set_value(i)


[ 0. nan nan]
[ 0.  0. nan]
[1. 0. 0.]
[2. 1. 0.]
[3. 2. 1.]


In [ ]:
#| export
class RMean(RValue): 
    """Calculates mean of the last n values"""
    def __init__(self, source: RValue, # source value
                 n:int # number of values to calculate the mean
                ): 
        super().__init__()
        self.last = RLastn(source, n, missing_val=np.nan)
    def calc(self): return self.last.value.mean()

In [ ]:
a = RInput(0)
v = RMean(a, 3)
for i in range(5): 
    print(v.value, v.last.value)
    a.set_value(i+1)


nan [ 0. nan nan]
nan [ 1.  0. nan]
1.0 [2. 1. 0.]
2.0 [3. 2. 1.]
3.0 [4. 3. 2.]


In [ ]:
#| export
class RLinRegression(RValue): 
    """Calculates linear regression for the source
       xs and ys should be of the same length.
    """
    def __init__(self, xs: RValue, # xs independent values
                 ys:RValue # ys dependent values
                ):
        super().__init__()
        store_attr()
        self.model = linear_model.LinearRegression()
    def calc(self): 
        in_x = self.xs.value.reshape(-1,1)
        in_y = self.ys.value
        model.fit(in_x, in_y)
        return self.model

    def predict(self, x): 
        """Given x, predicts next value y based on linear regression"""
        v = self.value.predict(np.array([[x]]))
        return v[0]

In [ ]:
show_doc(RLinRegression.predict)

---

### RLinRegression.predict

>      RLinRegression.predict (x)

Given x, predicts next value y based on linear regression

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
xs=np.array([1,2,3]).reshape(-1,1)
ys=np.array([0,5,4])
reg = linear_model.LinearRegression()
reg.fit(xs,ys)
reg.coef_, reg.predict(np.array([[5]]))

(array([2.]), array([9.]))